In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization, Dense, Activation, Dropout, Input, LSTM, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/share/software/user/open/py-scipy/1.6.3_py39/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "
2025-12-02 20:24:58.314541: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 20:24:58.476777: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-02 20:24:59.744066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.

## Scaling all sfe and climate data to be between 0 and 1
https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [2]:
# read in list of arrays for each pixel - these are fully filtered
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_filtered_window.npz", allow_pickle = True)["sfe_window"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_filtered_window.npz", allow_pickle = True)["climate_window"]



In [3]:
print(sfe_pre.shape)
print(climate_pre.shape)

(3799134,)
(3799134, 6, 8)


In [4]:
sfe_pre_rs = sfe_pre.reshape(-1,1) # single feature
sfe_pre_rs.shape


(3799134, 1)

In [5]:
# climate data is 3d, needs to be 2d for minmaxscaler
# so reshape and then turn back to original shape after processing
climate_pre_ogshape = climate_pre.shape
print(climate_pre_ogshape)

climate_pre_rs = climate_pre.reshape(-1, climate_pre_ogshape[2]) # (pixel-month pairs * timesteps, 8)
climate_pre_rs.shape

(3799134, 6, 8)


(22794804, 8)

In [6]:
scaler_sfe = MinMaxScaler(feature_range=(0,1))
scaler_climate = MinMaxScaler(feature_range=(0,1))

sfe_pre_scaled = scaler_sfe.fit_transform(sfe_pre_rs)
climate_pre_scaled = scaler_climate.fit_transform(climate_pre_rs)



In [7]:
print(sfe_pre_scaled.min(), sfe_pre_scaled.max())
print(climate_pre_scaled.min(), climate_pre_scaled.max())


0.0 1.0
0.0 1.0000000000000002


In [8]:
sfe_back = scaler_sfe.inverse_transform(sfe_pre_scaled)

sfe_back.shape
print(sfe_back.min(), sfe_back.max())
print(sfe_pre_rs.min(), sfe_pre_rs.max())


-0.4033722536155057 5.223653647385592
-0.40337225361550577 5.223653647385593


In [9]:
# change climate data back to orig shape
climate_pre_scaled = climate_pre_scaled.reshape(climate_pre_ogshape)

In [10]:
climate_pre_scaled = np.array(climate_pre_scaled)
sfe_pre_scaled = np.array(sfe_pre_scaled)

# wrap the lists as object arrays - allows arrays of different lengths to be saved
sfe_obj = np.array(sfe_pre_scaled, dtype=object)
climate_obj = np.array(climate_pre_scaled, dtype=object)

In [11]:
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_window_scaled.npz", sfe_pre = sfe_obj)
np.savez_compressed("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_window_scaled.npz", climate_pre =climate_obj)

## training the model

https://www.geeksforgeeks.org/deep-learning/long-short-term-memory-lstm-rnn-in-tensorflow/

In [12]:
# reading in sliding window data, was also scaled
sfe_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/sfe_window_scaled.npz", allow_pickle = True)["sfe_pre"] 
climate_pre = np.load("/scratch/users/ashdef/pre_treatment_data/no_nans/climate_window_scaled.npz", allow_pickle = True)["climate_pre"]

In [13]:
# converting from objects
sfe_pre = np.array(sfe_pre, dtype=np.float32)
climate_pre = np.array(climate_pre, dtype=np.float32)


In [14]:
climate_train, climate_test, sfe_train, sfe_test = train_test_split(
    climate_pre, sfe_pre, test_size=0.1, shuffle=False
)

In [15]:
model = Sequential()
model.add(LSTM(units=96, return_sequences=False,
          input_shape=(climate_pre.shape[1], climate_pre.shape[2]))) # return one output per sequence
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse', metrics = ['mae'])
model.summary()

2025-12-02 20:27:14.986906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-02 20:27:15.542420: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38199 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:c7:00.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 96)                40320     
                                                                 
 dropout (Dropout)           (None, 96)                0         
                                                                 
 dense (Dense)               (None, 1)                 97        
                                                                 
Total params: 40,417
Trainable params: 40,417
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(climate_train, sfe_train, epochs=20, batch_size=64, validation_split=0.1)

predictions = model.predict(climate_test)
np.save("/scratch/users/ashdef/model_out/6month_v3/predictions_scaled.npy", predictions)

predictions_original = scaler_sfe.inverse_transform(predictions)
np.save("/scratch/users/ashdef/model_out/6month_v3/predictions_original.npy", predictions_original)

np.save("/scratch/users/ashdef/model_out/6month_v3/sfe_test_scaled.npy", sfe_test)
    
sfe_test_original = scaler_sfe.inverse_transform(sfe_test)
np.save("/scratch/users/ashdef/model_out/6month_v3/sfe_test_original.npy", sfe_test_original)

model.save("/scratch/users/ashdef/model_out/6month_v3/baseline.keras")

rmse = mean_squared_error(sfe_test_original, predictions_original, squared=False)
mae = mean_absolute_error(sfe_test_original, predictions_original)
r2 = r2_score(sfe_test_original, predictions_original)

print(rmse, mae, r2)


Epoch 1/20


2025-12-02 20:27:18.792110: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


   11/48083 [..............................] - ETA: 4:09 - loss: 0.0599 - mae: 0.2001    

2025-12-02 20:27:19.397310: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


48083/48083 [==============================] - 246s 5ms/step - loss: 0.0024 - mae: 0.0357 - val_loss: 0.0017 - val_mae: 0.0304
Epoch 2/20
48083/48083 [==============================] - 245s 5ms/step - loss: 0.0017 - mae: 0.0301 - val_loss: 0.0014 - val_mae: 0.0270
Epoch 3/20
48083/48083 [==============================] - 242s 5ms/step - loss: 0.0014 - mae: 0.0280 - val_loss: 0.0013 - val_mae: 0.0269
Epoch 4/20
48083/48083 [==============================] - 248s 5ms/step - loss: 0.0013 - mae: 0.0267 - val_loss: 0.0013 - val_mae: 0.0263
Epoch 5/20
48083/48083 [==============================] - 247s 5ms/step - loss: 0.0012 - mae: 0.0258 - val_loss: 0.0012 - val_mae: 0.0250
Epoch 6/20
48083/48083 [==============================] - 246s 5ms/step - loss: 0.0011 - mae: 0.0251 - val_loss: 0.0013 - val_mae: 0.0256
Epoch 7/20
48083/48083 [==============================] - 248s 5ms/step - loss: 0.0011 - mae: 0.0246 - val_loss: 0.0011 - val_mae: 0.0239
Epoch 8/20
48083/48083 [=====================

In [17]:
history.history

{'loss': [0.0023885099217295647,
  0.0016568918945267797,
  0.001438477193005383,
  0.0012999997707083821,
  0.0012082081520929933,
  0.0011436725035309792,
  0.0010966808767989278,
  0.0010604243725538254,
  0.0010307994671165943,
  0.001007743994705379,
  0.0009862750303000212,
  0.0009681602823548019,
  0.0009538931190036237,
  0.0009421031572856009,
  0.0009318153606727719,
  0.0009204858215525746,
  0.0009107607766054571,
  0.000902220606803894,
  0.0008937648963183165,
  0.0008870938909240067],
 'mae': [0.03573344275355339,
  0.0300814937800169,
  0.028039459139108658,
  0.026685791090130806,
  0.025761956349015236,
  0.02508264221251011,
  0.024578474462032318,
  0.02417653053998947,
  0.023855628445744514,
  0.023599006235599518,
  0.023360883817076683,
  0.02315543219447136,
  0.02299392595887184,
  0.022849438712000847,
  0.02271977998316288,
  0.02260090969502926,
  0.02247893624007702,
  0.02238565869629383,
  0.02228153683245182,
  0.022193025797605515],
 'val_loss': [0.00

In [18]:
np.save("/scratch/users/ashdef/model_out/6month_v3/training_history.npy", history.history)